In [11]:
import requests
import re
from random import randint
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [28]:
#def data_film(genre='',sort='', rating='', subscript='available_online', country='', kind=''(film/serial), page=''):
def data_film(l):
    l = l.split('| ')
    url = f"https://www.kinopoisk.ru/lists/navigator/{l[0]}/?sort={l[1]}&quick_filters=%2C{l[2]}%2C{l[3]}%2C{l[4]}%2C{l[5]}&tab=online&page={l[6]}"
    dr = webdriver.Chrome()
    try:
        dr.get(url)
    except:
        return ([])
    print(url)
    soup = BeautifulSoup(dr.page_source)
    res = soup.find_all('div', {'class' :'desktop-seo-selection-film-item__upper-wrapper'})
    return (res)

In [13]:
def get_row(i, raw_data):
    name = raw_data[i].find_all('p', {'class' : 'selection-film-item-meta__name'}, limit = 1)
    if (len(name) > 0): name = name[0].text 
    else: return 0
    
    link = raw_data[i].find_all('a', {'class' : 'selection-film-item-meta__link'}, limit = 1)
    if (len(link) > 0): link = 'https://www.kinopoisk.ru' + link[0].get('href') 
    else: return 0
    
    date = raw_data[i].find_all('p', {'class' : 'selection-film-item-meta__original-name'}, limit = 1)
    if (len(date) > 0): date = date[0].text 
    else: return 0
    
    date = re.search(r'[\d]{4}$', date)
    if (date != None): date = date.group(0) 
    else: return 0
    
    x = raw_data[i].find_all('span', {'class' : 'selection-film-item-meta__meta-additional-item'}, limit = 2)
    if (len(x) > 1): country = x[0].text; gen = x[1].text 
    else: return 0
    
    rating = raw_data[i].find_all('span', {'class' : r'rating__value rating__value_positive'}, limit = 1)
    if (len(rating) == 0): rating = raw_data[i].find_all('span', {'class' : r'rating__value rating__value_neutral'}, limit = 1)
    if (len(rating) > 0): rating = rating[0].text 
    else: return 0
    
    rating_count = raw_data[i].find_all('span', {'class' : 'rating__count'}, limit = 1)
    if (len(rating_count) > 0): rating_count = rating_count[0].text 
    else: return 0
    
    return [name, date, country, gen, rating, rating_count, link]

In [14]:
def get_add_info(link):   
    dr = webdriver.Chrome()
    try:
        dr.get(link)
    except:
        return([])
    soup1 = BeautifulSoup(dr.page_source)
    info = soup1.find_all('div', {'class' : 'styles_valueLight__1j0RO'}, limit = 2)
    if (len(info) > 1): audio_track = info[0].text; subtitles = info[1].text
    else: return 0
    
    description = soup1.find_all('p', {'class' : 'styles_paragraph__2Otvx'}, limit = 1)
    if (len(description) > 0): description = description[0].text
    else: return 0

    return [audio_track, subtitles, description]

In [15]:
def get_ten(num, l):
    l += str((num - 1) // 50 + 1)
    num = (num - 1) % 50 
    raw_data = data_film(l)
    if (len(raw_data) == 0):
        return (0)
    df = pd.DataFrame(columns=['name','date','country','genre','rating','rating_count', 'link', 'audio', 'subtitles', 'description'])
    max_ind = len(raw_data)
    count = 0
    while (num < max_ind and count < 10):
        a = get_row(num , raw_data)
        if (a):
            a.extend([0, 0, 0])
            df.loc[count] = a
            count += 1
        num += 1
    return df    

In [62]:
#data_film(genre='',sort='', rating='', subscript='available_online', country='', kind=''(film/serial), page=''):
genre = ['', 'anime', 'mystery', 'comedy', 'crime', 'adventure', 'family', 'thriller', 'horror', 'sci-fi', 'fantasy']
sort = ['', 'popularity', 'year']
rating = ['', 'high_rated']
subscript= ['available_online', 'yandex_plus_subscription', 'kinopoisk_with_amediateka_subscription']
country = ['', 'foreign', 'russian']
kind = ['', 'films', 'serials']

In [29]:
#sort=year&quick_filters=films%2Cforeign%2Cyandex_plus_subscription
search_param = '| | | yandex_plus_subscription| foreign| | '
#{'genre': '', 'sort': 'year', 'rating': '', 'subscript': 'yandex_plus_subscription',\
 #               'country': 'foreign', 'kind': 'films', 'page': ''}
#data_film(genre='',sort='', rating='', subscript='available_online', country='', kind=''(film/serial), page=''):
data_ten = get_ten(51, search_param)

https://www.kinopoisk.ru/lists/navigator//?sort=&quick_filters=%2C%2Cyandex_plus_subscription%2Cforeign%2C&tab=online&page=2


In [30]:
s = '| | | yandex_plus_subscription| foreign| | '
s = s.split('| ')
print(s)

['', '', '', 'yandex_plus_subscription', 'foreign', '', '']


In [31]:
print(data_ten)

                                    name  date                  country  \
0                    Ford против Ferrari  2019                      США   
1                                Легенда  2015  Великобритания, Франция   
2                    Алита: Боевой ангел  2019              США, Япония   
3                                  Довод  2020      Великобритания, США   
4  Пираты Карибского моря: На краю света  2007                      США   
5                               Дэдпул 2  2018              США, Канада   
6                     Богемская рапсодия  2018      Великобритания, США   
7                    Корпорация монстров  2001                      США   
8                            Доктор Хаус  2012                      США   
9                         Великий Гэтсби  2013           Австралия, США   

                 genre rating rating_count  \
0     биография, спорт    8.2      397 341   
1    криминал, триллер    7.2      387 375   
2   фантастика, боевик    7.1      3

In [78]:
#df2 = df[df.name == '1+1']
df.loc[0]['audio'] = 'ads'
print(df.iloc[0])
#lst = get_add_info(df.loc[0]['link'])]

name                                          Не входи
date                                              2020
country                                 Великобритания
genre                                триллер, криминал
rating                                             5.3
rating_count                                    12 123
link            https://www.kinopoisk.ru/film/1251292/
audio                                              ads
subtitles                                            0
description                                          0
Name: 0, dtype: object


In [18]:
print('sss' == 'sss')

True
